# Quantum Computing with Julia

### Using module

As for now, include module's path and import it by "using ~".

In [1]:
include("./Juliaqat.jl")
using .Juliaqat

### Construct quantum circuit

Prepare input state.

In [2]:
input = Input(2) # Input(n) : n qubits |0> state

Input(2, Complex[0 + 0im, 0 + 0im], Gate[])

Prepare quantum gates as a list.

In [3]:
gates = [H(1), CX(1, 2)]
# gates = [H(1), H(2), CZ(1, 2), H(1), H(2)]
# gates = [H(1), H(2), CZ(1, 2), H(1), H(2), CZ(1, 2), H(1), H(2)]
# gates = [X(1), X(2), X(3)]

2-element Array{Gate,1}:
 H(1, "H")
 CX(1, 2, "CX")

Apply gates to input state.

In [4]:
apply!(input, gates)

Input(2, Complex[0 + 0im, 0 + 0im], Gate[H(1, "H"), CX(1, 2, "CX")])

Select a device.  
It is actually a simulator backend. Other simulators or hardware resources can be added to devices.

In [5]:
device = get_device("UndirectedGraph")

UndirectedGraphModel(true, [1])

As an option, some device dependent parameters can be contained in "device".  
As an example for "UndirectedGraph" simulator, if one of the computational basis states is given, it calculates output state amplitude only for the given basis state.

As default, "UndirectedGraph" simulator calculates full output state vector, it leads to $2^n$ loops.

In [6]:
# calculate amplitude for output state |11>.
# meas_output = [1, 1]
# device = get_device("UndirectedGraph", meas_output)

Quantum circuit execution.

In [7]:
state = execute(input, device)

4-element Array{Any,1}:
 0.7071067811865474 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
 0.7071067811865474 + 0.0im

In [8]:
device = get_device("StateVector")

StateVectorModel(true, Complex{Float64}[0.0 + 0.0im])

In [9]:
result = execute(input, device)

StateVectorResults(Any[Complex{Float64}[0.7071067811865476 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.7071067811865476 + 0.0im]], Any[[0, 0]])

In [10]:
input = Input(3)
gates = [H(1), CX(1, 2), CX(1, 3), M(1), M(2), M(3)]
apply!(input, gates)

Input(3, Complex[0 + 0im, 0 + 0im, 0 + 0im], Gate[H(1, "H"), CX(1, 2, "CX"), CX(1, 3, "CX"), M(1, 1, 0, "M"), M(2, 2, 0, "M"), M(3, 3, 0, "M")])

In [19]:
result = execute(input, device, 100)

StateVectorResults(Any[Complex{Float64}[0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 1.0000000000000002 + 0.0im], Complex{Float64}[0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 1.0000000000000002 + 0.0im], Complex{Float64}[0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 1.0000000000000002 + 0.0im], Complex{Float64}[0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 1.0000000000000002 + 0.0im], Complex{Float64}[0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 1.0000000000000002 + 0.0im], Complex{Float64}[1.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im], Complex{Float64}[1.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im], Complex{Float64}[1.0 + 0.0im, 0.0 + 0.0im, 0.0 +

In [20]:
get_counts(result)

Dict{String,Int64} with 2 entries:
  "000" => 42
  "111" => 58